# **Automatic Speech Recognition with Transformer**

##**Introduction**


- **Đề tài:** *Automatic Speech Recognition with Transformer* (Nhận dạng giọng nói tự động với Transformer)
- **Dữ liệu:** Gồm các file âm thanh được thu thập từ trang VOA: https://learningenglish.voanews.com 
Chuyển file âm thanh **.mp3 64kbps** sang âm thanh **Sample rate: 16k Hz, 1 Channel, wav**

    1.   Chia tất cả các file âm thanh thành câu. Đặt vào folder và bắt đầu từ số thứ tự là 1. Ví dụ: 
    Folder: African_Dinosaur_Discovered_in_Morocco

    /1.wav
    
    /2.wav

    /... 

    2.   Chia tất cả đoạn văn thành từng câu và xuống dòng. Đặt vào folder Story. Ví dụ: 

    Folder: African_Dinosaur_Discovered_in_Morocco

    /African_Dinosaur_Discovered_in_Morocco.txt
    3.   Mapping âm thanh và text với nhau.

- **Chia dữ liệu để Huấn luyện mô hình:**
    1.   Training Data: 80%
    2.   Evaluation Data: 20%

- **Mô hình:** Transformer


## **Data Preparation**

In [77]:
#Thư viện tính Word Error Rate
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [78]:
import os
import random
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras import layers
import natsort
import pandas as pd
import numpy as np
import librosa
from librosa.feature import mfcc
import warnings
warnings.filterwarnings("ignore")
from jiwer import wer

### **Download the dataset**

**Get link share từ Google Drive**

In [ ]:
#Điền link chứa Data
url = 'https://drive.google.com/file/d/1NNHHYIHn55l1P0hd7gm8Mq0oi0ugxlpl/view?usp=sharing'
id = url.split('/')[-2]

#Bỏ qua bước confirm nếu Google yêu cầu diệt virus
download_path = 'https://drive.google.com/uc?export=download&confirm=9iBg&id='+ id
print(download_path)

https://drive.google.com/uc?export=download&confirm=9iBg&id=1NNHHYIHn55l1P0hd7gm8Mq0oi0ugxlpl


**Tải file và giải nén bằng Keras**

In [ ]:
keras.utils.get_file(
    os.path.join(os.getcwd(), "Science&Technology.zip"),
    download_path,
    extract=True,
    archive_format="zip",
    cache_dir=".",
)

771203072/771190347 [==============================] - 10s 0us/step


'/content/Science&Technology.zip'

In [79]:
#Đường dẫn chứa thư mục gốc
dir = os.getcwd()

#Đường dẫn chứa file cần training
output_path = dir + "/datasets/Science&Technology/Output/"
print(output_path)

/content/datasets/Science&Technology/Output/


###**Preprocess the dataset**

In [80]:
def Get_Data():
    """
    Function objective: Chia data thành List chứa Audio_feature (Vector của âm thanh), label_audio (Label của âm thanh), text (từ trong âm thanh) từ Folder chứa file training, sau đó gắn vào list"
    Input: Đường dẫn chứa file âm thanh và file text
    Ouput: List chứa audio_list (Vector của âm thanh), label_list (Label của âm thanh), text_list (Từ của âm thanh)
    """
    audio_list = []                # Khởi tạo list chứa đường dẫnâm thanh
    text_list = []                 # Khởi tạo list chứa text ứng với âm thanh
    id_path_list = []              # Khởi tạo list chứa tên story và id audio

    story_count = 0                # Biến đếm số lượng story
    audio_count = 0                # Biến đếm số lượng file audio
    sentense_count = 0             # Biến đếm số lượng câu văn

    for story_folder in os.listdir(output_path):
      
        #Đếm số lượng Story
        story_count += 1

        #Sắp xếp các file theo thứ tự Alphabet
        alphabet = os.listdir(os.path.join(output_path, story_folder))
        file_sort = natsort.natsorted(alphabet)

        #Duyệt từng file đã sort
        for audio_file in file_sort:
            
            # Đường dẫn âm thanh
            audio_path = output_path + story_folder +'/'+ audio_file

            # Chỉ đọc file wav
            if audio_file.endswith("wav"):

                #Đếm số lượng file âm thanh
                audio_count += 1

                # Thêm đường dẫn của âm thanh vào list
                audio_list.append(audio_path)

                # Thêm id đường dẫn của âm thanh vào list
                id_path_list.append(story_folder + '/' + audio_file)

            # Chỉ đọc file text
            if audio_file.endswith("txt"):
                
                #Đọc file text
                f =  open(audio_path, "r", encoding="utf-8") 
                content = f.read()

                #Đọc từng câu trong file text 
                sentences = content.split("\n")

                for sentence in sentences:
                    
                    #Đếm số lượng câu văn
                    sentense_count += 1

                    #Thêm câu văn vào list
                    text_list.append(sentence)
            else: 
                continue
    print('Số lượng Story:', story_count)
    print('Số lượng Audio:', audio_count)
    print('Số lượng Câu:', audio_count)
    return audio_list, id_path_list, text_list    #Trả về list

**Đọc Data và hiển thị số lượng data**

In [81]:
audio_list, id_path_list, text_list = Get_Data()

Số lượng Story: 75
Số lượng Audio: 2078
Số lượng Câu: 2078


**Hiển thị data trên bảng**

In [82]:
#Đường dẫn chứa audio
df_audio = pd.DataFrame(columns=['Audio_Path'])
df_audio['Audio_Path'] = audio_list

#Đường dẫn chứa id audio
df_path = pd.DataFrame(columns=['ID_Path'])
df_path['ID_Path'] = id_path_list

#Text ứng với audio
df_text = pd.DataFrame(columns=['Text'])
df_text['Text'] = text_list

#Hiển thị data
df_data = pd.concat([df_audio, df_path, df_text, ], axis=1)
df_data

,Audio_Path,ID_Path,Text
0,/content/datasets/Science&Technology/Output/Lu...,Lunar_Eclipse_to_Darken_Moon_over_North_South_...,People in many parts of North and South Americ...
1,/content/datasets/Science&Technology/Output/Lu...,Lunar_Eclipse_to_Darken_Moon_over_North_South_...,A lunar eclipse
2,/content/datasets/Science&Technology/Output/Lu...,Lunar_Eclipse_to_Darken_Moon_over_North_South_...,happens when the moon passes through Earth's s...
3,/content/datasets/Science&Technology/Output/Lu...,Lunar_Eclipse_to_Darken_Moon_over_North_South_...,as our planet goes around the Sun
4,/content/datasets/Science&Technology/Output/Lu...,Lunar_Eclipse_to_Darken_Moon_over_North_South_...,This kind of event does not happen every year
...,...,...,...
2073,/content/datasets/Science&Technology/Output/Li...,Listening_to_a_Spider_Web/23.wav,"There are more than 47,000 kinds of spiders."
2074,/content/datasets/Science&Technology/Output/Li...,Listening_to_a_Spider_Web/24.wav,They all create these webs to provide housing ...
2075,/content/datasets/Science&Technology/Output/Li...,Listening_to_a_Spider_Web/25.wav,"Scientists say silk, the material created to f..."
2076,/content/datasets/Science&Technology/Output/Li...,Listening_to_a_Spider_Web/26.wav,Buehler said that understanding the living str...


**Mapping Audio và Text**

In [83]:
def Generate_Data(audio_list, id_path_list, text_list):
  """
  Function objective: Nối âm thanh ứng với text
  Input: List chứa đường dẫn file âm thanh, id âm thanh, text của âm thanh
  Ouput: Data đã được mapping giữa audio và text
  """
  path_to_text = {}

  #Duyệt từng phần tử trong id_path
  for i in range(len(id_path_list)):      #Duyệt từng phần tử trong id_path
      text = text_list[i]                 #Lấy text tương ứng
      id = id_path_list[i]                #Lấy ID từ id_path
      path_to_text[id] = text             #Gắn text và id path

  data = []
  for w in audio_list:                                      #Duyệt từng đường dẫn file audio
      id = w.split("/")[-2] + "/" + w.split("/")[-1]        #Cắt id từ đường dẫn
      if len(path_to_text[id]) < 200:                       #Text ứng với id < hơn 200 từ thì mapping audio và text
        data.append({"audio": w, "text": path_to_text[id]}) #Thêm vào data
  return data                                               #Trả về kết quả

**Trả về Data đã được Mapping**

In [84]:
data = Generate_Data(audio_list, id_path_list, text_list)
print("Data đã được mapping:", len(data))

Data đã được mapping: 2064


**Chuẩn hóa data**

In [85]:
#Chuyển text tự về dạng vector
class VectorizeChar:
    def __init__(self, max_len=50):
        self.vocab = (
            ["-", "#", "<", ">"]
            + [chr(i + 96) for i in range(1, 27)]
            + [" ", ".", ",", "?"]
        )
        self.max_len = max_len
        self.char_to_idx = {}
        for i, ch in enumerate(self.vocab):
            self.char_to_idx[ch] = i

    def __call__(self, text):
        text = text.lower()
        text = text[: self.max_len - 2]
        text = "<" + text + ">"
        pad_len = self.max_len - len(text)
        return [self.char_to_idx.get(ch, 1) for ch in text] + [0] * pad_len

    def get_vocabulary(self):
        return self.vocab

#Output < 200 kí tự
max_target_len = 200  
vectorizer = VectorizeChar(max_target_len)
print("Số từ vựng:", len(vectorizer.get_vocabulary()))

#Khởi tạo data set của text
def create_text_ds(data):
    """
    Function objective: Khởi tạo data set chứa text
    Input: data đã được mapping. Type(list)
    Ouput: List chứa text dạng tensor
    """
    texts = [_["text"] for _ in data]
    text_ds = [vectorizer(t) for t in texts]
    text_ds = tf.data.Dataset.from_tensor_slices(text_ds)
    return text_ds


def path_to_audio(path):
    """
    Function objective: Lấy vector đặc trưng của audio
    Input: Đường dẫn chứa audio. Type(string)
    Ouput: Vector đặc trưng của âm thanh. 
    """
    #Chuyển audio thành dạng spectrogram sử dụng stft
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1)
    audio = tf.squeeze(audio, axis=-1)
    stfts = tf.signal.stft(audio, frame_length=200, frame_step=80, fft_length=256)
    x = tf.math.pow(tf.abs(stfts), 0.5)

    # Chuẩn hóa audio
    means = tf.math.reduce_mean(x, 1, keepdims=True)
    stddevs = tf.math.reduce_std(x, 1, keepdims=True)
    x = tf.math.divide_no_nan(x-means, stddevs)
    # x = (x - means) / stddevs
    audio_len = tf.shape(x)[0]

    # padding audio về 10s
    pad_len = 2500
    paddings = tf.constant([[0, pad_len], [0, 0]])
    x = tf.pad(x, paddings, "CONSTANT")[:pad_len, :]
    return x

#Khởi tạo data set của audio
def create_audio_ds(data):
    """
    Function objective: Khởi tạo data set chứa vector audio
    Input: data đã được mapping. Type(list)
    Ouput: Vector đặc trưng của âm thanh dạng tensor 
    """
    flist = [_["audio"] for _ in data]
    audio_ds = tf.data.Dataset.from_tensor_slices(flist)
    audio_ds = audio_ds.map(
        path_to_audio, num_parallel_calls=tf.data.AUTOTUNE
    )
    return audio_ds

#Khởi tạo data set của cả audio và text
def create_tf_dataset(data, bs=4):
    """
    Function objective: Khởi tạo data set chứa vector audio và text
    Input: data đã được mapping - Type(list), batchsize của dữ liệu - type(int)
    Ouput: dataset chứa vector audio và text dạng tensor
    """
    audio_ds = create_audio_ds(data)
    text_ds = create_text_ds(data)
    ds = tf.data.Dataset.zip((audio_ds, text_ds))
    ds = ds.map(lambda x, y: {"source": x, "target": y})
    ds = ds.batch(bs)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


Số từ vựng: 34


**Tách data**
- Training: **80%**
- Evaluation: **20%**

In [86]:
#Tách data theo yêu cầu đề bài
split = int(len(data) * 0.80)
train_data = data[:split]
test_data = data[split:]

#Gọi lại hàm khởi tại trước đó
ds = create_tf_dataset(train_data, bs=16)
val_ds = create_tf_dataset(test_data, bs=16)

print("Train:", len(ds))
print("Evaluation:", len(val_ds))

Train: 104
Evaluation: 26


## **Training the model**
Model sử dụng: **Transformer**

###Define the Transformer Input Layer

In [87]:
class TokenEmbedding(layers.Layer): 
    def __init__(self, num_vocab=1000, maxlen=100, num_hid=64):
        super().__init__()
        #Tạo mã thông báo gồm các vectơ có kích thước cố định.
        self.emb = tf.keras.layers.Embedding(num_vocab, num_hid) 

        #Tạo vị trị mã thông báo ( inject thêm thông tin về vị trí của một từ)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid) 

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        x = self.emb(x)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions

class SpeechFeatureEmbedding(layers.Layer): 
    def __init__(self, num_hid=64, maxlen=100):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv2 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.conv3 = tf.keras.layers.Conv1D(
            num_hid, 11, strides=2, padding="same", activation="relu"
        )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=num_hid)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return self.conv3(x)

###Transformer Encoder Layer

In [89]:
class TransformerEncoder(layers.Layer):
    """
    Function objective: Lớp được mã hóa
    Input: Layer đã nhúng, số lượng headatention, layer ffd
    Ouput: Layer được mã hóa,  đầu vào cho bộ giải mã.
    """
    def __init__(self, embed_dim, num_heads, feed_forward_dim, rate=0.1):
        super().__init__()
        #Tạo chiếu truy vấn, khóa và giá trị cho Multi-head Attention
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        #Tạo mô hình tuần tự (chuyển danh sách các lớp đến phương thức khởi tạo Tuần tự)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

###Transformer Decoder Layer

In [90]:
class TransformerDecoder(layers.Layer):
    """
    Function objective: Lớp được giải hóa
    Input: Layer đã nhúng, số lượng headatention, layer ffd, tỉ lệ dropout
    Ouput: Layer được giải mã
    """
    def __init__(self, embed_dim, num_heads, feed_forward_dim, dropout_rate=0.1):
        super().__init__()
        # Tạo lớp Normalization chuẩn hóa lại đầu ra của multi-head attention
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.self_att = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.enc_att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        #Dropout để các unit bị ẩn đi trong quá trình training làm giảm tính dự đoán của model
        self.self_dropout = layers.Dropout(0.5)
        self.enc_dropout = layers.Dropout(0.1)
        self.ffn_dropout = layers.Dropout(0.1)
        self.ffn = keras.Sequential(
            [
                layers.Dense(feed_forward_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        """Khi Decoder dịch đến từ thứ i, phần sau sẽ bị che lại (masked) và 
        Decoder chỉ được phép nhìn thấy phần nó đã dịch trước đó.
        Để ngăn chặn luồng thông tin từ mã thông báo trong tương lai đến mã thông báo hiện tại
        """
        i = tf.range(n_dest)[:, None]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        mask = tf.cast(m, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, enc_out, target):
        input_shape = tf.shape(target)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        target_att = self.self_att(target, target, attention_mask=causal_mask)
        target_norm = self.layernorm1(target + self.self_dropout(target_att))
        enc_out = self.enc_att(target_norm, enc_out)
        enc_out_norm = self.layernorm2(self.enc_dropout(enc_out) + target_norm)
        ffn_out = self.ffn(enc_out_norm)
        ffn_out_norm = self.layernorm3(enc_out_norm + self.ffn_dropout(ffn_out))
        return ffn_out_norm

###Complete the Transformer model

In [91]:
class Transformer(keras.Model):
    """
    Function objective: Tạo mô hình Transformer
    Input: Hidden layer, head-attention, feed_forward, số lượng từ source, số lượng từ đích, layer mã hóa và giải mả, num class.
    Ouput: Layer được giải mã
    """
    def __init__(
        self,
        num_hid=64,
        num_head=2,
        num_feed_forward=128,
        source_maxlen=100,
        target_maxlen=100,
        num_layers_enc=4,
        num_layers_dec=1,
        num_classes=10,
    ):
        super().__init__()
        self.loss_metric = keras.metrics.Mean(name="loss")
        self.num_layers_enc = num_layers_enc
        self.num_layers_dec = num_layers_dec
        self.target_maxlen = target_maxlen
        self.num_classes = num_classes
        self.enc_input = SpeechFeatureEmbedding(num_hid=num_hid, maxlen=source_maxlen)
        self.dec_input = TokenEmbedding(
            num_vocab=num_classes, maxlen=target_maxlen, num_hid=num_hid
        )
        self.encoder = keras.Sequential(
            [self.enc_input]
            + [
                TransformerEncoder(num_hid, num_head, num_feed_forward)
                for _ in range(num_layers_enc)
            ]
        )

        for i in range(num_layers_dec):
            setattr(
                self,
                f"dec_layer_{i}",
                TransformerDecoder(num_hid, num_head, num_feed_forward),
            )

        self.classifier = layers.Dense(num_classes)

    def decode(self, enc_out, target):
        y = self.dec_input(target)
        for i in range(self.num_layers_dec):
            y = getattr(self, f"dec_layer_{i}")(enc_out, y)
        return y

    def call(self, inputs):
        source = inputs[0]
        target = inputs[1]
        x = self.encoder(source)
        y = self.decode(x, target)
        return self.classifier(y)

    @property
    def metrics(self):
        return [self.loss_metric]

    def train_step(self, batch):
        """Processes one batch inside model.fit()."""
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        with tf.GradientTape() as tape:
            preds = self([source, dec_input])
            one_hot = tf.one_hot(dec_target, depth=self.num_classes)
            mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
            loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def test_step(self, batch):
        source = batch["source"]
        target = batch["target"]
        dec_input = target[:, :-1]
        dec_target = target[:, 1:]
        preds = self([source, dec_input])
        one_hot = tf.one_hot(dec_target, depth=self.num_classes)
        mask = tf.math.logical_not(tf.math.equal(dec_target, 0))
        loss = self.compiled_loss(one_hot, preds, sample_weight=mask)
        self.loss_metric.update_state(loss)
        return {"loss": self.loss_metric.result()}

    def generate(self, source, target_start_token_idx):
        """đầu vào sử dụng greedy decoding."""
        bs = tf.shape(source)[0]
        enc = self.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(self.target_maxlen - 1):
            dec_out = self.decode(enc, dec_input)
            logits = self.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input

 ### Callbacks to display predictions

In [92]:
class DisplayOutputs(keras.callbacks.Callback):
    def __init__( #Tạo các lệnh gọi lại
        self, batch, idx_to_token, target_start_token_idx=27, target_end_token_idx=28
    ):
        """Hiển thị kết quả đầu ra sau mỗi epoch

        Args:
             batch: test chứa các khóa "nguồn" và "đích"
             idx_to_token: Danh sách chứa các mã thông báo từ vựng tương ứng với các chỉ số của chúng
             target_start_token_idx: Chỉ mục mã thông báo bắt đầu trong từ vựng đích
             target_end_token_idx: Chỉ mục mã thông báo kết thúc trong từ vựng đích
        """
        self.batch = batch
        self.target_start_token_idx = target_start_token_idx
        self.target_end_token_idx = target_end_token_idx
        self.idx_to_char = idx_to_token

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        source = self.batch["source"]
        target = self.batch["target"].numpy()
        bs = tf.shape(source)[0]
        preds = self.model.generate(source, self.target_start_token_idx)
        preds = preds.numpy()
        for i in range(bs):
            target_text = "".join([self.idx_to_char[_] for _ in target[i, :]])
            prediction = ""
            for idx in preds[i, :]:
                prediction += self.idx_to_char[idx]
                if idx == self.target_end_token_idx:
                    break
            wer_score = wer(target_text, prediction)    #Tính toán WER
            print("-" * 100)
            print(f"Word Error Rate: {wer_score:.4f}")
            print("-" * 100)
            print(f"target:     {target_text.replace('-','')}")
            print(f"prediction: {prediction}\n")
            

### Learning rate schedule

In [93]:
class CustomSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self,
        init_lr=0.00001,
        lr_after_warmup=0.001,
        final_lr=0.00001,
        warmup_epochs=15,
        decay_epochs=85,
        steps_per_epoch=203,
    ):
        super().__init__()
        self.init_lr = init_lr
        self.lr_after_warmup = lr_after_warmup
        self.final_lr = final_lr
        self.warmup_epochs = warmup_epochs
        self.decay_epochs = decay_epochs
        self.steps_per_epoch = steps_per_epoch

    def calculate_lr(self, epoch):
        """ linear warm up - linear decay """
        warmup_lr = (
            self.init_lr
            + ((self.lr_after_warmup - self.init_lr) / (self.warmup_epochs - 1)) * epoch
        )
        decay_lr = tf.math.maximum(
            self.final_lr,
            self.lr_after_warmup
            - (epoch - self.warmup_epochs)
            * (self.lr_after_warmup - self.final_lr)
            / (self.decay_epochs),
        )
        return tf.math.minimum(warmup_lr, decay_lr)

    def __call__(self, step):
        epoch = step // self.steps_per_epoch
        return self.calculate_lr(epoch)


In [94]:
batch = next(iter(val_ds))

# Tạo từ vựng để chuyển đổi các chỉ số dự đoán thành các ký tự
idx_to_char = vectorizer.get_vocabulary()
display_cb = DisplayOutputs(
    batch, idx_to_char, target_start_token_idx=2, target_end_token_idx=3
)  # đặt các đối số trong '<' và '>'

#Khởi tạo mô hình
model = Transformer(
    num_hid=128,
    num_head=2,
    num_feed_forward=128,
    target_maxlen=200,
    num_layers_enc=4,
    num_layers_dec=1,
    num_classes=34,
)

#Hàm mất mát
loss_fn = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True, label_smoothing=0.1,
)

#Khởi tạo learning rate
learning_rate = CustomSchedule(
    init_lr=0.00001,
    lr_after_warmup=0.001,
    final_lr=0.00001,
    warmup_epochs=15,
    decay_epochs=85,
    steps_per_epoch=len(ds),
)

#Khởi tạo hàm tối ưu
optimizer = keras.optimizers.Adam(learning_rate)

#Compile Mô hình
model.compile(optimizer=optimizer, loss=loss_fn)

## **Evaluation**

In [95]:
history = model.fit(ds, validation_data=val_ds, callbacks=[display_cb], epochs=1)

104/104 [==============================] - ETA: 0s - loss: 1.4104----------------------------------------------------------------------------------------------------
Word Error Rate: 1.5000
----------------------------------------------------------------------------------------------------
target:     <until now, huge ichthyosaurs had not been known to have lived at the end of the triassic period>
prediction: <iaoncrererererereexer g#a   tatns xeq taeq sst   tstntg o o tasru g nde g o  oi g httcd nas o  heerd  g as ttieeud  geerersna tgpsreeree g g>

----------------------------------------------------------------------------------------------------
Word Error Rate: 2.0769
----------------------------------------------------------------------------------------------------
target:     <scientists had thought the group disappeared from earth a few million years earlier>
prediction: <iaoncrererererereexer g#e   tatns xeq taeq sst   tstntg o o tasru g nde g o  oi g httcd nas o  heerd  g as

In [96]:
model.summary()

Model: "transformer_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 speech_feature_embedding_6   (None, None, 128)        542464    
 (SpeechFeatureEmbedding)                                        
                                                                 
 token_embedding_6 (TokenEmb  multiple                 29952     
 edding)                                                         
                                                                 
 sequential_49 (Sequential)  (None, None, 128)         1204480   
                                                                 
 transformer_decoder_15 (Tra  multiple                 297728    
 nsformerDecoder)                                                
                                                                 
 dense_94 (Dense)            multiple                  4386      
                                                     

## **Demonstration**